In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
# If 0, the code will run on CPU; if >=1, Colab likely has a GPU (e.g. T4) available.


Num GPUs Available: 1


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1️⃣ Read the data
df = pd.read_csv('df_subset_20.csv')

# 2️⃣ Clean the title column (remove NaN or convert them to empty strings)
titles = df['title_clean'].fillna('')  # replaces NaN with empty string

# 3️⃣ Create and fit the vectorizer
tfidf = TfidfVectorizer(max_features=500)

# 4️⃣ Transform titles
X_text = tfidf.fit_transform(titles).toarray()  # (n_samples, 500)
print(X_text.shape)



(533295, 500)


In [3]:
missing_titles = df[df['title_clean'].isna()]
print("Missing titles:", len(missing_titles))


Missing titles: 2


In [4]:
from sklearn.preprocessing import OneHotEncoder

cat = df[['category']]
ohe = OneHotEncoder(sparse_output=False)  # use sparse_output instead of sparse
X_cat = ohe.fit_transform(cat)
print(X_cat.shape)  # (n_samples, n_categories)



(533295, 266)


In [5]:
from sklearn.preprocessing import OneHotEncoder

cat = df[['category']]
ohe = OneHotEncoder(sparse_output=False)   # ✅ correct for sklearn >= 1.2
X_cat = ohe.fit_transform(cat)

print(X_cat.shape)  # (n_samples, n_categories)




(533295, 266)


In [6]:
import sklearn
print(sklearn.__version__)


1.6.1


In [7]:
import numpy as np
X = np.hstack([X_text, X_cat])  # final input features


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_price_train, y_price_test, y_flag_train, y_flag_test = train_test_split(
    X, df['price'], df['isBestSeller'], test_size=0.2, random_state=42)


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define model architecture
input_shape = X_train.shape[1]  # total number of features (text + category)
inputs = Input(shape=(input_shape,), name='product_input')
x = Dense(128, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)

# Regression output (price)
price_output = Dense(1, name='price_output')(x)   # linear by default

# Classification output (bestseller flag)
bestseller_output = Dense(1, activation='sigmoid', name='flag_output')(x)

model = Model(inputs=inputs, outputs=[price_output, bestseller_output])

# Compile with appropriate losses and metrics for each task
model.compile(optimizer='adam',
              loss={'price_output': 'mse', 'flag_output': 'binary_crossentropy'},
              metrics={'price_output': 'mae', 'flag_output': 'accuracy'})


In [10]:
# Prepare target dictionaries as required by Keras
y_train_dict = {'price_output': y_price_train.values,
                'flag_output':  y_flag_train.values}
y_test_dict  = {'price_output': y_price_test.values,
                'flag_output':  y_flag_test.values}

# Train for a few epochs
history = model.fit(X_train, y_train_dict,
                    validation_data=(X_test, y_test_dict),
                    epochs=20, batch_size=4)


Epoch 1/20
106659/106659 ━━━━━━━━━━━━━━━━━━━━ 276s 3ms/step - flag_output_accuracy: 0.9936 - flag_output_loss: 0.0393 - loss: 0.0396 - price_output_loss: 3.6670e-04 - price_output_mae: 0.0124 - val_flag_output_accuracy: 0.9940 - val_flag_output_loss: 0.0382 - val_loss: 0.0384 - val_price_output_loss: 1.7051e-04 - val_price_output_mae: 0.0095
Epoch 2/20
106659/106659 ━━━━━━━━━━━━━━━━━━━━ 272s 3ms/step - flag_output_accuracy: 0.9938 - flag_output_loss: 0.0373 - loss: 0.0376 - price_output_loss: 3.0337e-04 - price_output_mae: 0.0104 - val_flag_output_accuracy: 0.9940 - val_flag_output_loss: 0.0334 - val_loss: 0.0335 - val_price_output_loss: 1.2886e-04 - val_price_output_mae: 0.0064
Epoch 3/20
106659/106659 ━━━━━━━━━━━━━━━━━━━━ 272s 3ms/step - flag_output_accuracy: 0.9938 - flag_output_loss: 0.0368 - loss: 0.0370 - price_output_loss: 2.6757e-04 - price_output_mae: 0.0093 - val_flag_output_accuracy: 0.9940 - val_flag_output_loss: 0.0395 - val_loss: 0.0396 - val_price_output_loss: 9.7863e-05

In [11]:
losses = model.evaluate(X_test, y_test_dict)
print("Test results:", losses)


3334/3334 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - flag_output_accuracy: 0.9943 - flag_output_loss: 0.0331 - loss: 0.0333 - price_output_loss: 1.2710e-04 - price_output_mae: 0.0057
Test results: [0.03521221503615379, 0.00012749037705361843, 0.03507595136761665, 0.9940370917320251, 0.005714695435017347]


In [12]:
# Average price per category
avg_price_by_cat = df.groupby('category')['price'].mean()

# Average monthly sales per category
avg_sales_by_cat = df.groupby('category')['boughtInLastMonth'].mean()

# Bestseller ratio per category (percentage)
bestseller_ratio = df.groupby('category')['isBestSeller'].mean() * 100


In [13]:
# === Inference utilities ===
import numpy as np

def _prep_single(title: str, category: str):
    # match training preproc: lowercase & strip for safety
    t = [str(title).strip().lower()]
    c = np.array([[str(category).strip()]])  # 2D for OneHotEncoder
    Xt = tfidf.transform(t).toarray()
    Xc = ohe.transform(c)                    # uses the *fitted* ohe
    Xn = np.hstack([Xt, Xc])
    return Xn

def predict_one(title: str, category: str, thr: float = 0.5):
    Xn = _prep_single(title, category)
    pred_price, pred_flag = model.predict(Xn, verbose=0)
    price = float(pred_price[0][0])
    prob  = float(pred_flag[0][0])
    is_best = (prob >= thr)
    return price, prob, is_best


In [21]:
sample_title = ("uxcell M6 x 1.0 Spiral Flute Thread Tap, Metric Machine Threading Tap HSS Nitriding Coated, Round Shank with Square End, H2 Tolerance, 2pcs")
sample_category = "Hand Tools"   # change if your true category label is different

p_price, p_prob, p_isbest = predict_one(sample_title, sample_category)
print("— DEMO —")
print("Category:", sample_category)
print(f"Predicted Price: ${p_price:,.2f}")
print(f"Bestseller Probability: {p_prob:.3f}")
print("Bestseller?:", "YES ✅" if p_isbest else "NO ❌")


— DEMO —
Category: Hand Tools
Predicted Price: $0.01
Bestseller Probability: 0.001
Bestseller?: NO ❌


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [15]:
def predict_batch(titles: list, categories: list, thr: float = 0.5):
    Xt = tfidf.transform([t.strip().lower() for t in titles]).toarray()
    Xc = ohe.transform(np.array([[c] for c in categories]))
    Xn = np.hstack([Xt, Xc])
    pred_price, pred_flag = model.predict(Xn, verbose=0)
    probs = pred_flag.ravel()
    isbest = (probs >= thr).astype(int)
    return pred_price.ravel(), probs, isbest

# Example usage:
# prices, probs, flags = predict_batch(df.sample(5)['title_clean'].tolist(),
#                                      df.sample(5)['category'].tolist())


In [16]:
from sklearn.metrics import classification_report, mean_absolute_error

# classification metrics
y_prob = model.predict(X_test, verbose=0)[1].ravel()
y_hat  = (y_prob >= 0.5).astype(int)
print("\n— Bestseller (flag) on holdout —")
print(classification_report(y_flag_test.astype(int), y_hat, digits=3))

# regression MAE
y_pred_price = model.predict(X_test, verbose=0)[0].ravel()
mae = mean_absolute_error(y_price_test, y_pred_price)
print(f"— Price MAE on holdout: {mae:,.4f}")



— Bestseller (flag) on holdout —
              precision    recall  f1-score   support

           0      0.994     1.000     0.997    106023
           1      0.000     0.000     0.000       636

    accuracy                          0.994    106659
   macro avg      0.497     0.500     0.499    106659
weighted avg      0.988     0.994     0.991    106659



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


— Price MAE on holdout: 0.0057


In [17]:
import joblib, os
os.makedirs("artifacts", exist_ok=True)

model.save("artifacts/mtl_ffnn.keras")     # Keras model
joblib.dump(tfidf, "artifacts/tfidf.pkl")  # text vectorizer
joblib.dump(ohe,   "artifacts/ohe.pkl")    # category encoder

print("Saved to ./artifacts")


Saved to ./artifacts
